In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
# Libraries
import os
import pickle

from src.models import train_model, test_model, predict_model

In [25]:
# os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

In [26]:
EXPERIMENT = 'mh09'
MODEL = 'model_A1'
EXP_DIR = f'./models/{EXPERIMENT}/'
# Training
PATH_TO_DATA = f'data/processed/synthetic_data/{MODEL}/n1000/'
# Models
model_template = f'./models/templates/{MODEL}_noisyor.pl'

In [27]:
fractions = [0.1, 1, 5, 10, 20]
mutate = ['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7']

In [6]:
for frac in fractions:
    for m in mutate:
        path_to_data = f'{PATH_TO_DATA}mutate_{m}_{frac}/'
        train_model.main(
            experiment = EXPERIMENT, 
            path_to_data = path_to_data, 
            model_template = model_template, 
            training = 0.8, 
            max_iter = 20, 
            evidence = ['activity', 'occupancy', 'fold_change'],
            facts = [],
            suffix = f'_{m}_{frac}'
        )

Splitting data...
Starting LFI...
problog lfi ./models/templates/model_A1_noisyor.pl ./models/mh09/evidence_m1_0.1.pl -O ./models/mh09/model_trained_m1_0.1.pl --normalize -k ddnnf -v -n 20
[INFO] Output level: INFO
[INFO] Number of examples: 800
[INFO] Compile time: 35.729953
[INFO] Weights to learn: [t(0.33,P)::fold_change(P,S,inc), t(0.33,P)::fold_change(P,S,dec), t(0.33,P)::fold_change(P,S,baseline), t(0.33,E)::activity(E,S,inc), t(0.33,E)::activity(E,S,dec), t(0.33,E)::activity(E,S,baseline), t(0.33,_fc)::occupancy(P,S,inc), t(0.33,_fc)::occupancy(P,S,dec), t(0.33,_fc)::occupancy(P,S,baseline), t(0.33,E,_dM)::occupancy(P,S,inc), t(0.33,E,_dM)::occupancy(P,S,dec), t(0.33,E,_dM)::occupancy(P,S,baseline)]
[INFO] Bodies: [lfi_body(0,t(P),P,S,inc), lfi_body(1,t(P),P,S,dec), lfi_body(2,t(P),P,S,baseline), lfi_body(3,t(E),E,S,inc), lfi_body(4,t(E),E,S,dec), lfi_body(5,t(E),E,S,baseline), lfi_body(6,t(_fc),P,S,inc), lfi_body(7,t(_fc),P,S,dec), lfi_body(8,t(_fc),P,S,baseline), lfi_body(9,t(

In [28]:
# Testing
test = ['training', 'testing']
PATH_TO_TRUTH = f'data/processed/synthetic_data/{MODEL}/n1000/'
results = {}

In [ ]:
for frac in fractions:
    for m in mutate:
        model_trained = f'{EXP_DIR}model_trained_{m}_{frac}.pl'
        results[f'{m}_{frac}'] = {}
        for test_on in test:
            PATH_TO_TESTING = f'{EXP_DIR}{test_on}_{m}_{frac}/'
            compare_to = [test_on, 'truth']
            for compare in compare_to:
                if compare == 'truth':
                    path_to_truth = PATH_TO_TRUTH
                else: 
                    path_to_truth = PATH_TO_TESTING

                test_results = test_model.main(
                    experiment = EXPERIMENT, 
                    path_to_data = PATH_TO_TESTING, 
                    path_to_truth = path_to_truth, 
                    model_trained = model_trained, 
                    targets = ['occupancy', 'modification'], 
                    test_on = f'{test_on}_{m}_{frac}', compare = f'{compare}_{m}_{frac}',
                    mode='inference')
                results[f'{m}_{frac}'][f'{test_on}_vs_{compare}'] = test_results

In [30]:
predictions = {}
for frac in fractions:
    for m in mutate:
        predictions[f'{m}_{frac}'] = {}
        PATH_TO_TESTING = f'{EXP_DIR}testing_{m}_{frac}/'
        model_trained = f'{EXP_DIR}model_trained_{m}_{frac}.pl'
        predictions[f'{m}_{frac}'] = predict_model.main(
            experiment = EXPERIMENT, 
            path_to_data = PATH_TO_TESTING, 
            model_trained = model_trained, 
            targets = ['occupancy', 'modification'],
            mode = 'inference', infer_from = 'facts', 
            suffix = f'_{m}_{frac}_testing')
results['predictions'] = predictions

Inferring data (s10)...
problog ./models/mh09/make_predictions.pl -o ./models/mh09/predictions/inferred_example.txt
Finished inference (s10)...
Inferring data (s100)...
problog ./models/mh09/make_predictions.pl -o ./models/mh09/predictions/inferred_example.txt
Finished inference (s100)...
Inferring data (s101)...
problog ./models/mh09/make_predictions.pl -o ./models/mh09/predictions/inferred_example.txt
Finished inference (s101)...
Inferring data (s109)...
problog ./models/mh09/make_predictions.pl -o ./models/mh09/predictions/inferred_example.txt
Finished inference (s109)...
Inferring data (s117)...
problog ./models/mh09/make_predictions.pl -o ./models/mh09/predictions/inferred_example.txt
Finished inference (s117)...
Inferring data (s12)...
problog ./models/mh09/make_predictions.pl -o ./models/mh09/predictions/inferred_example.txt
Finished inference (s12)...
Inferring data (s126)...
problog ./models/mh09/make_predictions.pl -o ./models/mh09/predictions/inferred_example.txt
Finished in

In [31]:
with open(f'{EXP_DIR}results.pickle', 'wb') as file:
    pickle.dump(results, file, protocol=pickle.HIGHEST_PROTOCOL)